In [ ]:
import collections
import pickle
import pandas as pd
import os
import peppi_py
import tree
import json
import io
import math
import tqdm.notebook
import functools
import zipfile

from slippi_db import utils, preprocessing, parse_peppi

In [ ]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

In [ ]:
root = '/linusr/vlad/SSBM/Replays'

In [ ]:
parsed_path = os.path.join(root, 'parsed.pkl')

with open(parsed_path, 'rb') as f:
    data_bytes = f.read()
len(data_bytes)

In [ ]:
%%time
data = pickle.loads(data_bytes)
del data_bytes
len(data)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
@functools.cache
def compact_raw(raw: str) -> str:
    if raw.startswith('Players/'):
        return raw.split('/')[1]
    if raw.startswith('Phillip/'):
        return 'Phillip'
    return raw

df['compact_raw'] = df['raw'].map(compact_raw)

In [ ]:
@functools.cache
def raw_exists(raw: str) -> bool:
    if not raw.endswith('.zip'):
        return False
    return os.path.exists(os.path.join(root, 'Raw', raw))

df['raw_exists'] = df['raw'].map(raw_exists)

In [ ]:
def get_files_in_zip(path: str) -> list[str]:
    filenames = []
    with zipfile.ZipFile(path, mode='r') as zf:
        for info in zf.infolist():
            if info.is_dir():
                continue
            filenames.append(info.filename)
    return set(filenames)

In [ ]:
raw_to_files = {}
for raw in tqdm.notebook.tqdm(set(df['raw'])):
    if not raw.endswith('.zip'):
        continue
    if not raw_exists(raw):
        continue

    raw_to_files[raw] = get_files_in_zip(os.path.join(root, 'Raw', raw))

In [ ]:
missing_files = {}

for raw, name in zip(df['raw'], df['name']):
    if raw not in raw_to_files:
        continue
    if name not in raw_to_files[raw]:
        missing_files.setdefault(raw, []).append(name)

assert not missing_files

In [ ]:
file_to_raw = {}
for raw, files in raw_to_files.items():
    for f in files:
        # File could be in two different dumps, one for each player
        file_to_raw[f] = raw

In [ ]:
# Fix bad raws.
# Could be slightly wrong for files that exist in multiple raws,
# which may be slightly different due to different rollback frames.

for row in data:
    raw = row['raw']
    if raw not in raw_to_files:
        continue
    name = row['name']
    if name not in raw_to_files[raw]:
        row['raw'] = file_to_raw[name]

In [ ]:
# %%time

# with open(os.path.join(root, 'parsed.pkl'), 'wb') as f:
#     pickle.dump(data, f)

In [ ]:
valid = df[df['valid']]
training = valid[valid['is_training']]
len(training)

In [ ]:
training_with_raw = training[training['raw_exists']]
len(training_with_raw)

In [ ]:
for_test_dataset = training_with_raw.groupby('raw').tail(50).reset_index()
size = len(for_test_dataset)
size

In [ ]:
files_to_copy = {}
for raw, filename in zip(for_test_dataset['raw'], for_test_dataset['name']):
    files_to_copy.setdefault(raw, []).append(filename)

In [ ]:
dest_zip = os.path.join(root, f'TestDataset-{size}.zip')

if os.path.exists(dest_zip):
    os.remove(dest_zip)

sources_and_files = [
    (os.path.join(root, 'Raw', raw), filenames)
    for raw, filenames in files_to_copy.items()
]

utils.copy_multi_zip_files(sources_and_files, dest_zip)

In [ ]:
all_raw = set(df['raw'])

In [ ]:
player_raws = {}
for raw in all_raw:
    if not raw.startswith('Players/'):
        continue
    player = raw.split('/')[1]
    player_raws.setdefault(player, []).append(raw)


print(','.join(sorted(player_raws.keys())))